In [1]:
def is_running_in_colab():
    try:
        from google.colab import userdata
        print("Running in Google Colab")
        return True
    except ImportError:
        print("Running in local environment")
        return False

In [2]:
if is_running_in_colab():
    # %%capture
    !pip install unsloth
    !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Running in Google Colab
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-2p9d9co3
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-2p9d9co3
  Resolved https://github.com/unslothai/unsloth.git to commit d1d15f1d14f1168837d29b9c08e9b6d63945d469
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.2.12-py3-none-any.whl size=187179 sha256=903d75f0d9bdd709a56f8d4ca8beca601f42384254e2de97c2dbfa94d503635f
  Stored in directory: /tmp/pip-ephem-wheel-cache-swglkic4/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.2.12
    Uninstalling unsloth-2025.2.12:
      Successfully uninstalled unsloth-2025.2.12


In [4]:
if is_running_in_colab():
    from google.colab import userdata
    HUGGINGFACE_TOKEN = userdata.get("HUGGINGFACE_TOKEN")
else:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

Running in Google Colab


In [13]:
from unsloth import FastLanguageModel

mex_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=mex_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
    token=HUGGINGFACE_TOKEN
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1536: UserWarning: Current model requires 4194304.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

#Fine-tuning

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.2.12 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [9]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a chess expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

# Testing

In [10]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a chess expert with advanced knowledge in strategy, tactics, positional play, and endgame theory.
Please analyze the following chess position or question and provide a detailed response.

### Question:
{}
Analyze this chess position by addressing:
1. Material and piece placement
2. Control of key squares and pawn structure
3. Your calculated main line (2-3 moves deep)
4. Strategic plans for both sides
Then, on a new line, recommend the best move based on this analysis.
Write the best move in UCI notation (e.g., e2e4, e7e5, e1g1, e7e8q...)

### Response:
<think>{}"""

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and construct a step-by-step reasoning process to ensure a logical and accurate response.

### Instruction:
You are a chess expert with advanced knowledge in strategy, tactics, positional play, and endgame theory.
Please analyze the following chess position or question and provide a detailed response.

### Question:
{}

### Analysis:
1. **Material and piece placement**:
   - Describe the current material balance and the positioning of key pieces.
2. **Control of key squares and pawn structure**:
   - Identify key squares controlled by each side and how the pawn structure influences the game.
3. **Main line calculation (2-3 moves deep)**:
   - Provide a calculated variation with reasoning.
4. **Strategic plans for both sides**:
   - Suggest key plans and possible maneuvers.

### Best Move:
Based on the above analysis, the best move is:
**UCI Notation:** {}
"""

In [11]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and construct a step-by-step reasoning process to ensure a logical and accurate response.

### Instruction:
K, Q, R, B, N, P are the notations for the white king, queen, rook, bishop, knight, and pawn respectively.
The notation for the black pieces are the same but with lowercased letters. A period represents an empty square.
Please analyze the following chess position:

r . b q k b n r
p p . . p p p p
. . n p . . . .
. B p . . . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K . . R

Side to move: White

Castling rights: White can castle kingside

En passant target square: None

### Analysis:
1. Material and piece placement:
   - Describe the current material balance and the positioning of key pieces.
2. Control of key squares and pawn structure:
   - Identify key squares controlled by each side and how the pawn structure influences the game.
3. Main line calculation (2-3 moves deep):
   - Provide a calculated variation with reasoning.
4. Strategic plans for both sides:
   - Suggest key plans and possible maneuvers.

### Best Move:
Based on the above analysis, return the best move in UCI notation (e.g., e2e4, e7e5, e1g1, e7e8q...):
UCI Notation: [fill in here]
"""

In [12]:
question = "Black to move. Position is 8/8/3k4/R7/8/n2K4/8/8 b - - 18 70. What is the best move?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=10000,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
# print(response[0].split("### Best Move:")[1])
print(response[0])

<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and construct a step-by-step reasoning process to ensure a logical and accurate response.

### Instruction:
K, Q, R, B, N, P are the notations for the white king, queen, rook, bishop, knight, and pawn respectively.
The notation for the black pieces are the same but with lowercased letters. A period represents an empty square.
Please analyze the following chess position:

r . b q k b n r
p p . . p p p p
. . n p . . . .
. B p . . . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K . . R

Side to move: White

Castling rights: White can castle kingside

En passant target square: None

### Analysis:
1. Material and piece placement:
   - Describe the current material balance and the positioning of key pieces.
2. Control of key squares and pawn structur